# Cleaning and visualizing your data

No dataset is perfect. Much as with human sources, your data won't have the complete story. It will be able to point you in a new direction or supply valuable context or contain a surprising insight. And it will be representative of the flaws in the process and the people that created it.

How to account for all that? Cleaning. No dataset is usable until it's gone through a process by which you know it's as clean as you can get it -- and the ways in which it's still imperfect. Communicating that to editors and audience is important, too.

Some things to look for when you're diving into at a dataset:

- Are there blank rows at the beginning or end of the file?
- What does each column header mean? What are the values you'd expect to find in that column?
- Are all the columns importing with the proper column type? Strings, datetimes, floats, integers.
- Are there `NaN` (or `null` or `''` or `None`) values in any of your columns? Are there any columns with mostly `NaN` values?
- Is there duplicate data?
- Are there fields that appear to have been entered inconsistently? Pay attention to capitalization and punctuation!
- Do you see any funky special characters?

Much of the time spent working with data will be with a scouring pad, a mop, and sometimes a toothbrush.

Occasionally you get to use a flamethrower. Those are both good and bad days.

Once you have your data, the first trick is usually figuring out which tool you need next.


One of the first things that leaps out is we have an `Unnamed: 6` column that seems to contain ... nothing. That's what it looks like, so we should check and make sure.

Perfect. There's nothing in it but nothin'. Let's get rid of it.

Now let's look again and make sure that did what we think it did.

Lovely. Onward.

Next, let's take a look at this `facid` column. It appears to be a unique id, which is always helpful to have when dealing with a dataset. So let's see if it really is unique.

Hm. This is worth looking at more closely. What `dtype: int64` is telling us is that `pandas` wants to treat this as a column of integers -- which it clearly isn't. There are letters mixed in and we have no reason to try to do math on these or anything.

Let's make sure these come in as strings instead.

Cool. `O` is for object, which in `pandas` is what we're looking for.

Now take another look at the output from `value_counts()`. We want to use these as unique IDs but it looks like there's a value in that column twice. We should take a closer look at that.

We've done a littler bit of filtering, so let's try that again here.

Yep, we've got a duplicate. One seems to have more complete data, so that's the one we want to keep. We'll drop the other one.

We're going to drop the row we don't want by selecting its index number, 165, and then resetting the index so that it's numbered sequentially again. For safety, we're also going to write the result to a new variable so that, if we mess up and have to rerun this cell, we won't run the risk of dropping any other row that gets numbered 165.

Speaking of messing up: In your Pandas adventures, there's probably going to be a point where you've got a lot of variables saved and things just aren't doing what you think they should be doing. There's an easy way to start fresh in those cases — just select Kernel>Restart & Clear Output to restart so you can run everything from the beginning, or select Kernel>Restart & Run All to clear all variables and rerun your whole notebook.

Now let's filter again to make sure we no longer have a duplicate.

Fan-bloody-tastic. Now that we can be pretty confident we don't have duplicates, we can turn out attention to that combined `city_state_zip` column. It's _almost_ helpful. Surely we'd prefer to look at these facilities by city or zip code separately, so we're going to want them in separate columns.

The last five characters of each `city_state_zip` entry seem to be our zip codes, so we'll `slice` those off on their own.

And now we can look again to make sure we did that correctly and none of the values are wonky.

How does that look to y'all? Anything stand out?

Format-wise, this actually worked out pretty well — looks like there weren't any items in that column entered wrong, so all of these are actually zip codes. As an added bonus, it's always best to read zip codes in as string or categorical data, since integer or float formats will drop the leading zeros. And that is its own hellscape.

Speaking of leading zeros, though, that 05400 zipcode doesn't quite look right. From the context in the rest of the data, we can be pretty sure that's not an Oregon zip code.

Let's take a closer look with a quick filter.

AHA! Someone was messing with our data. (Full disclosure: It was me.)

If it was a little less fake-looking, we might double-check the `facid` against the complaints dataset to make sure it doesn't match any entries there.

So now we know that we don't need this as we continue through the cleaning and analysis, we can kill it.

And so let's look again at those zip codes.

A note on zip codes: We got a little lucky having to separate out the zip codes from an existing string. That's not always the case, and `pandas` often will see them as integers -- if our dataset had had a separate column of zip codes, there's a 99 percent chance they would have imported as numbers. Especially in New England, where most zip codes begin with a zero, we end up with four-digit zip codes in our data -- which, y'know ... it's not good. [Here's how you can import a column of zip codes as strings]('http://data-tutorials.com/zip-codes-in-pandas.html').

So! On to city names.

Our city names go up to the first comma, so we can use that to split the string into what comes before and what comes after that comma. Then we can grab the first item and that should be our city name.

So let's have a look.

Those look pretty good. We can move on to looking at the `owner` and `operator` fields. Those will be easier to look at quickly if we use a built-in Python function to sort them.

Right off the bat, we can see there are some duplicates. A change in capitalization, a change in punctuation, a change in spacing -- any of these can cause this. So to the degree that we can, we should try to standardize the names here.

First, let's get a sense of the size of the universe of names we're looking at.

That's about 700 things we'd have to check manually to resolve differences -- more than enough to know code might be able to help us narrow down this normalization problem.

What we need are some rules we can apply to all of these to try to eliminate the sublte differences. We can make everything uppercase, for example, and we can remove all the commas and periods.

So let's do that.

Now let's look at that `operator` column again.

Yo. That is an improvement.

We can take another look at the size of our universe and get a sense of how we've narrowed things down.

Better than having to do that by hand, certainly.

OK. So now let's reintroduce our complaints data.

One of those is not like the other. Quite often, because data is as messy as humans are, you'll see dates unrealistically in the past or the future. Sometimes it's a result of how the system that stores the data handles empty values or poorly formatted historical data. And sometimes it's just, like, because.

In this case, let's briefly peer into the future.

It's one record, and this is a quick analysis, which means this is a reasonable case to make a note that we tossed this record out and move on. 

Feels like this is in pretty good shape. We've looked at a lot of the initial things to check, so a little analysis and a chart or two seem like a good thing to work toward from here.

And thus we return to our good friend `groupby()` for some summaries.

Let's borrow the merge from the previous session and add the complaint rate to our data

## And now we become data artists

First, some tools for drawing the data.

Brilliant. We made a chart.

HOLY JEEZ WE MADE A CHART.

As we might expect, a quick visualization gives us more questions to ask: 
- Why so few reports pre-2010?
- Are 2010 and 2016 really abberations, or is this partial data for some reason?

If you're doing this analysis at work, this is one place you're probably going to want to circle back to your data source or another expert to find out exactly how this was collected. (We're, um. We're not going to do that today.)

Instead, let's filter those older records so we can get a better picture of complaints since 2010 and throw a title on that chart. And we might very want to show someone else what we've discovered, so we'll export it to a file to make that easy.

Way better.

Now let's take a look visually at those complaint ratios.

These are not the prettiest charts ever -- and they don't have to be. They're another tool for quickly analyzing and understanding your data. And if you're working with a designer or want to plug your results into another visualization engine, this can help you decide the best tool to use or the best representation to make.

If you're looking to up your visualization game, check out Python libraries [Altair]('https://altair-viz.github.io/') and [seaborn]('https://seaborn.pydata.org/'), which will give you prettier static charts right out of the box, or [bokeh]('https://bokeh.pydata.org/en/latest/') for interactive graphics.